Prosseguimos o desenvolvimento do trabalho elaborando um estudo de simulações a partir do conjunto de imagens Fashion-MNIST. Para cada classe de roupa (10 classes) atribuímos um valor específico para $\theta(x)$. Deste modo, esperamos que o modelo seja capaz de diferenciar os tipos de imagens e estimar o parâmetro associado a elas corretamente.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import random
from time import time

from scipy.special import comb, loggamma, lambertw

import mps
import pwexp

In [2]:
# Valor máximo do suporte da distribuição
B = 10001

# ---------------------------- Poisson (Pocr) ----------------------------
# Versões sem o tensorflow
log_a_poisson = lambda m : -loggamma(m+1)
log_phi_poisson = lambda theta : np.log(theta)
C_poisson = lambda theta : np.exp(theta)
C_inv_poisson = lambda u : np.log(u)
# Versões para o tensorflow
log_a_poisson_tf = lambda m : -tf.math.lgamma(m+1)
log_phi_poisson_tf = lambda theta : tf.math.log(theta)
C_poisson_tf = lambda theta : tf.math.exp(theta)
C_inv_poisson_tf = lambda u : tf.math.log(u)
sup_poisson = np.arange(0, B, 1).astype(np.float64)

theta_min_poisson = None
theta_max_poisson = None
def E_poisson(theta):
    return theta
# Variance is always equal to the mean
def Var_poisson(theta):
    return theta

# ---------------------------- Logarithmic (Locr) ----------------------------
# Versões sem o tensorflow
log_a_log = lambda m : -np.log(m+1)
log_phi_log = lambda theta : np.log(theta)
C_log = lambda theta : -np.log(1-theta)/theta
C_inv_log = lambda u : 1 + np.real(lambertw(-u*np.exp(-u))) / u
# Versões para o tensorflow
log_a_log_tf = lambda m : -tf.math.log(m+1)
log_phi_log_tf = lambda theta : tf.math.log(theta)
C_log_tf = lambda theta : -tf.math.log(1-theta)/theta
C_inv_log_tf = lambda u : 1 + tfp.math.lambertw(-u*tf.math.exp(-u)) / u
sup_log = np.arange(0, B, 1).astype(np.float64)

theta_min_log = 0
theta_max_log = 1
def E_log(theta):
    return -theta / (np.log(1-theta)*(1-theta)) - 1
# Overdispersion: in this case, variance is always greater than mean
def Var_log(theta):
    return -theta*(theta + np.log(1-theta)) / ((1-theta)**2*(np.log(1-theta))**2)

# ---------------------------- Binomial Negativa (+Geométrica) (NBcr + Gecr) ----------------------------
# Versões sem o tensorflow
def log_a_nb(q):
    return lambda m : loggamma(m+q) - loggamma(m+1) - loggamma(q)
def log_phi_nb(q):
    return lambda theta : np.log(1-theta)
def C_nb(q):
    return lambda theta : theta**(-q)
def C_inv_nb(q):
    return lambda u : u**(-1/q)
# Versões para o tensorflow
def log_a_nb_tf(q):
    return lambda m : tf.math.lgamma(m+q) - tf.math.lgamma(m+1) - tf.math.lgamma(q)
def log_phi_nb_tf(q):
    return lambda theta : tf.math.log(1-theta)
def C_nb_tf(q):
    return lambda theta : theta**(-q)
def C_inv_nb_tf(q):
    return lambda u : u**(-1/q)
sup_nb = np.arange(0, B, 1).astype(np.float64)

theta_min_nb = 0
theta_max_nb = 1
def E_nb(q, theta):
    return q*(1-theta)/theta
# Overdispersion: in this case, variance is always greater than mean
def Var_nb(q, theta):
    return q*(1-theta)/theta**2

# ---------------------------- Binomial (+Bernoulli) (Bincr + Bercr) ----------------------------
# Versões sem o tensorflow
def log_a_bin(q):
    return lambda m : loggamma(q+1) - loggamma(m+1) - loggamma(q-m+1)
def log_phi_bin(q):
    return lambda theta : np.log(theta) - np.log(1-theta)
def C_bin(q):
    return lambda theta : (1-theta)**(-q)
def C_inv_bin(q):
    return lambda u : 1 - u**(-1/q)
# Versões para o tensorflow
def log_a_bin_tf(q):
    return lambda m : tf.math.lgamma(q+1) - tf.math.lgamma(m+1) - tf.math.lgamma(q-m+1)
def log_phi_bin_tf(q):
    return lambda theta : tf.math.log(theta) - tf.math.log(1-theta)
def C_bin_tf(q):
    return lambda theta : (1-theta)**(-q)
def C_inv_bin_tf(q):
    return lambda u : 1 - u**(-1/q)
def sup_bin(q):
    return np.arange(0, q+1, 1).astype(np.float64)

theta_min_bin = 0
theta_max_bin = 1
def E_bin(q, theta):
    return q*theta
# Underdispersion: in this case, variance is always lesser than mean
def Var_bin(q, theta):
    return q*theta*(1-theta)

# ---------------------------- Borel (Bocr) ----------------------------
# Versões sem o tensorflow
log_a_borel = lambda m : (m-1)*np.log(m+1) - loggamma(m+1)
log_phi_borel = lambda theta : np.log(theta) - theta
C_borel = lambda theta : np.exp(theta)
C_inv_borel = lambda u : np.log(u)
# Versões para o tensorflow
log_a_borel_tf = lambda m : (m-1)*tf.math.log(m+1) - tf.math.lgamma(m+1)
log_phi_borel_tf = lambda theta : tf.math.log(theta) - theta
C_borel_tf = lambda theta : tf.math.exp(theta)
C_inv_borel_tf = lambda u : tf.math.log(u)
sup_borel = np.arange(0, B, 1).astype(np.float64)

theta_min_borel = 0
theta_max_borel = 1
def E_borel(theta):
    return theta/(1-theta)
# Overdispersion: in this case, variance is always greater than mean
def Var_borel(theta):
    mu = theta/(1-theta)
    return mu*(1+mu)**2

# ---------------------------- Restricted Generalized Poisson (RGPcr) ----------------------------
# Versões sem o tensorflow
def log_a_rgp(q):
    return lambda m : (m-1)*np.log(1+q*m) - loggamma(m+1)
def log_phi_rgp(q):
    return lambda theta : np.log(theta) - q*theta
def C_rgp(q):
    return lambda theta : np.exp(theta)
def C_inv_rgp(q):
    return lambda u : np.log(u)
# Versões para o tensorflow
def log_a_rgp_tf(q):
    return lambda m : (m-1)*tf.math.log(1+q*m) - tf.math.lgamma(m+1)
def log_phi_rgp_tf(q):
    return lambda theta : tf.math.log(theta) - q*theta
def C_rgp_tf(q):
    return lambda theta : tf.math.exp(theta)
def C_inv_rgp_tf(q):
    return lambda u : tf.math.log(u)
def sup_rgp(q):
    if(q > 0):
        return np.arange(0, B, 1).astype(np.float64)
    else:
        if(q < -1):
            raise Exception("q value can't be less than -1")
        max_sup_candidates = np.arange(1, 101)
        max_sup = max_sup_candidates[(1 + max_sup_candidates*q) > 0][-1]
        return np.arange(max_sup+1)

theta_min_rgp = 0
# The RGP theta parameter must be lesser than q, otherwise its probabilities do not sum to one
def theta_max_rgp(q):
    return np.abs(1/q)
def E_rgp(q, theta):
    return theta/(1-q*theta)
# Overdispersion: in this case, variance is always greater than mean
def Var_rgp(q, theta):
    mu = theta/(1 - q*theta)
    return mu / (1-q*theta)**2


# ---------------------------- Haight (Catalan) (Cacr) - Geeta(q = 2) ----------------------------
# Versões sem o tensorflow
log_a_haight = lambda m : loggamma(2*m+2) - loggamma(m+2) - loggamma(m+1) - np.log(2*m+1)
log_phi_haight = lambda theta : np.log(theta) + np.log(1-theta)
C_haight = lambda theta : 1/(1-theta)
C_inv_haight = lambda u : 1 - 1/u
# Versões para o tensorflow
log_a_haight_tf = lambda m : tf.math.lgamma(2*m+2) - tf.math.lgamma(m+2) - tf.math.lgamma(m+1) - tf.math.log(2*m+1)
log_phi_haight_tf = lambda theta : tf.math.log(theta) + tf.math.log(1-theta)
C_haight_tf = lambda theta : 1/(1-theta)
C_inv_haight_tf = lambda u : 1 - 1/u
sup_haight = np.arange(0, B, 1).astype(np.float64)

theta_min_haight = 0
theta_max_haight = 0.5
def E_haight(theta):
    p = theta
    s = 1-p
    return s/(s-p) - 1
# Overdispersion: in this case, variance is always greater than mean
def Var_haight(theta):
    p = theta
    s = 1-p
    return p*s/(s-p)**2 + 2*p**2*s/(s-p)**3

# ---------------------------- Geeta (Gecr) ----------------------------
# Versões sem o tensorflow
def log_a_geeta(q):
    return lambda m : loggamma(q*m+q) - loggamma(m+2) - loggamma((q-1)*m+q-1) - np.log(q*m+q-1)
def log_phi_geeta(q):
    return lambda theta : np.log(theta) + (q-1)*np.log(1-theta)
def C_geeta(q):
    return lambda theta : (1-theta)**(1-q)
def C_inv_geeta(q):
    return lambda u : 1 - u**(1/(1-q))
# Versões para o tensorflow
def log_a_geeta_tf(q):
    return lambda m : tf.math.lgamma(q*m+q) - tf.math.lgamma(m+2) - tf.math.lgamma(q*m+q) - tf.math.log(q*m+q-1)
def log_phi_geeta_tf(q):
    return lambda theta : tf.math.log(theta) + (q-1)*tf.math.log(1-theta)
def C_geeta_tf(q):
    return lambda theta : 1/(1-theta)
def C_inv_geeta_tf(q):
    return lambda u : 1 - u**(1/(1-q))
sup_geeta = np.arange(0, B, 1).astype(np.float64)

theta_min_geeta = 0
theta_max_geeta = lambda q : 1/q
def E_geeta(q, theta):
    p = theta
    s = 1-p
    return s/(s-p*(q-1)) - 1
def Var_geeta(q, theta):
    p = theta
    s = 1-p
    return p*s*(q-1)/(s-p*(q-1))**2 + p**2*s*(q-1)*q/(s-p*(q-1))**3

In [3]:
def sim_dist(log_a, log_phi, sup, theta, n):
    x = mps.rvs(log_a, log_phi, theta, sup, size = n)
    x_mean = np.mean(x)
    x_var = np.var(x)
    print("    Sample mean: {:.6f}".format(x_mean))
    print("    Sample var: {:.6f}".format(x_var))
    
def sim_dist_thetas(log_a, log_phi, sup, thetas, n, E_func, Var_func, q = None):
    for theta in thetas:
        print("--- theta = {:.4f}".format(theta))
        if(q is None):
            print("    Theoretical mean: {:.6f}".format(E_func(theta)))
            print("    Theoretical variance: {:.6f}".format(Var_func(theta)))
        else:
            print("    Theoretical mean: {:.6f}".format(E_func(q, theta)))
            print("    Theoretical variance: {:.6f}".format(Var_func(q, theta)))
        sim_dist(log_a, log_phi, sup, theta, n)

In [4]:
n = 500000

## Poisson distribution

In [5]:
print("Poisson distribution")
sim_dist_thetas(log_a_poisson, log_phi_poisson, sup_poisson, [0.5, 1, 2, 5, 10, 20], n, E_poisson, Var_poisson, q = None)

Poisson distribution
--- theta = 0.5000
    Theoretical mean: 0.500000
    Theoretical variance: 0.500000
    Sample mean: 0.500354
    Sample var: 0.499980
--- theta = 1.0000
    Theoretical mean: 1.000000
    Theoretical variance: 1.000000
    Sample mean: 1.000274
    Sample var: 0.998534
--- theta = 2.0000
    Theoretical mean: 2.000000
    Theoretical variance: 2.000000
    Sample mean: 2.004108
    Sample var: 2.002043
--- theta = 5.0000
    Theoretical mean: 5.000000
    Theoretical variance: 5.000000
    Sample mean: 4.999392
    Sample var: 5.001808
--- theta = 10.0000
    Theoretical mean: 10.000000
    Theoretical variance: 10.000000
    Sample mean: 10.002260
    Sample var: 10.019887
--- theta = 20.0000
    Theoretical mean: 20.000000
    Theoretical variance: 20.000000
    Sample mean: 20.002222
    Sample var: 19.991521


## Logarithmic distribution

In [6]:
print("Logarithmic distribution")
sim_dist_thetas(log_a_log, log_phi_log, sup_log, [0.1, 0.25, 0.5, 0.75, 0.9], n, E_log, Var_log, q = None)

Logarithmic distribution
--- theta = 0.1000
    Theoretical mean: 0.054580
    Theoretical variance: 0.059616
    Sample mean: 0.054690
    Sample var: 0.059539
--- theta = 0.2500
    Theoretical mean: 0.158686
    Theoretical variance: 0.202361
    Sample mean: 0.158306
    Sample var: 0.200597
--- theta = 0.5000
    Theoretical mean: 0.442695
    Theoretical variance: 0.804021
    Sample mean: 0.442810
    Sample var: 0.806885
--- theta = 0.7500
    Theoretical mean: 1.164043
    Theoretical variance: 3.973090
    Sample mean: 1.162280
    Sample var: 3.970145
--- theta = 0.9000
    Theoretical mean: 2.908650
    Theoretical variance: 23.808956
    Sample mean: 2.918074
    Sample var: 23.824042


## NB(q = 1) distribution (Geometric distribution)

In [7]:
print("Geometric distribution")
sim_dist_thetas(log_a_nb(1), log_phi_nb(1), sup_nb, [0.1, 0.25, 0.5, 0.75, 0.9], n, E_nb, Var_nb, q = 1)

Geometric distribution
--- theta = 0.1000
    Theoretical mean: 9.000000
    Theoretical variance: 90.000000
    Sample mean: 8.994090
    Sample var: 89.983487
--- theta = 0.2500
    Theoretical mean: 3.000000
    Theoretical variance: 12.000000
    Sample mean: 2.996706
    Sample var: 11.935847
--- theta = 0.5000
    Theoretical mean: 1.000000
    Theoretical variance: 2.000000
    Sample mean: 0.996738
    Sample var: 1.994723
--- theta = 0.7500
    Theoretical mean: 0.333333
    Theoretical variance: 0.444444
    Sample mean: 0.334084
    Sample var: 0.447292
--- theta = 0.9000
    Theoretical mean: 0.111111
    Theoretical variance: 0.123457
    Sample mean: 0.110502
    Sample var: 0.123019


In [34]:
# theta = (1-p)/p, p being the probability for the geometric
print("Mean-Variance Negative Binomial distribution (q = 1) = Geometric")
sim_dist_thetas(log_a_mvnb(1), log_phi_mvnb(1), sup_nb, [9, 3, 1, 1/3, 1/9], n, E_mvnb, Var_mvnb, q = 1)

Mean-Variance Negative Binomial distribution (q = 1) = Geometric
--- theta = 9.0000
    Theoretical mean: 9.000000
    Theoretical variance: 90.000000
    Sample mean: 8.988450
    Sample var: 89.581705
--- theta = 3.0000
    Theoretical mean: 3.000000
    Theoretical variance: 12.000000
    Sample mean: 3.002762
    Sample var: 12.053678
--- theta = 1.0000
    Theoretical mean: 1.000000
    Theoretical variance: 2.000000
    Sample mean: 0.998474
    Sample var: 1.998284
--- theta = 0.3333
    Theoretical mean: 0.333333
    Theoretical variance: 0.444444
    Sample mean: 0.334230
    Sample var: 0.444800
--- theta = 0.1111
    Theoretical mean: 0.111111
    Theoretical variance: 0.123457
    Sample mean: 0.111380
    Sample var: 0.123526


## NB(q = 2) distribution

In [8]:
print("Geometric distribution")
sim_dist_thetas(log_a_nb(2), log_phi_nb(2), sup_nb, [0.1, 0.25, 0.5, 0.75, 0.9], n, E_nb, Var_nb, q = 2)

Geometric distribution
--- theta = 0.1000
    Theoretical mean: 18.000000
    Theoretical variance: 180.000000
    Sample mean: 18.012232
    Sample var: 180.142866
--- theta = 0.2500
    Theoretical mean: 6.000000
    Theoretical variance: 24.000000
    Sample mean: 6.001322
    Sample var: 24.076656
--- theta = 0.5000
    Theoretical mean: 2.000000
    Theoretical variance: 4.000000
    Sample mean: 1.999324
    Sample var: 4.008872
--- theta = 0.7500
    Theoretical mean: 0.666667
    Theoretical variance: 0.888889
    Sample mean: 0.666834
    Sample var: 0.889718
--- theta = 0.9000
    Theoretical mean: 0.222222
    Theoretical variance: 0.246914
    Sample mean: 0.222268
    Sample var: 0.247701


## Bin(q = 1) distribution (Bernoulli)

In [9]:
print("Bernoulli distribution")
sim_dist_thetas(log_a_bin(1), log_phi_bin(1), sup_bin(1), [0.1, 0.25, 0.5, 0.75, 0.9], n, E_bin, Var_bin, q = 1)

Bernoulli distribution
--- theta = 0.1000
    Theoretical mean: 0.100000
    Theoretical variance: 0.090000
    Sample mean: 0.100636
    Sample var: 0.090508
--- theta = 0.2500
    Theoretical mean: 0.250000
    Theoretical variance: 0.187500
    Sample mean: 0.250886
    Sample var: 0.187942
--- theta = 0.5000
    Theoretical mean: 0.500000
    Theoretical variance: 0.250000
    Sample mean: 0.499682
    Sample var: 0.250000
--- theta = 0.7500
    Theoretical mean: 0.750000
    Theoretical variance: 0.187500
    Sample mean: 0.748906
    Sample var: 0.188046
--- theta = 0.9000
    Theoretical mean: 0.900000
    Theoretical variance: 0.090000
    Sample mean: 0.899502
    Sample var: 0.090398


## Bin(q = 20) distribution

In [10]:
print("Bernoulli distribution")
sim_dist_thetas(log_a_bin(20), log_phi_bin(20), sup_bin(20), [0.1, 0.25, 0.5, 0.75, 0.9], n, E_bin, Var_bin, q = 20)

Bernoulli distribution
--- theta = 0.1000
    Theoretical mean: 2.000000
    Theoretical variance: 1.800000
    Sample mean: 1.998714
    Sample var: 1.797328
--- theta = 0.2500
    Theoretical mean: 5.000000
    Theoretical variance: 3.750000
    Sample mean: 4.999336
    Sample var: 3.745988
--- theta = 0.5000
    Theoretical mean: 10.000000
    Theoretical variance: 5.000000
    Sample mean: 10.001474
    Sample var: 5.007372
--- theta = 0.7500
    Theoretical mean: 15.000000
    Theoretical variance: 3.750000
    Sample mean: 14.997052
    Sample var: 3.748787
--- theta = 0.9000
    Theoretical mean: 18.000000
    Theoretical variance: 1.800000
    Sample mean: 18.001736
    Sample var: 1.799061


## Borel distribution

In [11]:
print("Borel distribution")
sim_dist_thetas(log_a_borel, log_phi_borel, sup_borel, [0.1, 0.25, 0.5, 0.75, 0.9], n, E_borel, Var_borel, q = None)

Borel distribution
--- theta = 0.1000
    Theoretical mean: 0.111111
    Theoretical variance: 0.137174
    Sample mean: 0.111788
    Sample var: 0.137851
--- theta = 0.2500
    Theoretical mean: 0.333333
    Theoretical variance: 0.592593
    Sample mean: 0.332142
    Sample var: 0.588536
--- theta = 0.5000
    Theoretical mean: 1.000000
    Theoretical variance: 4.000000
    Sample mean: 1.004718
    Sample var: 4.043948
--- theta = 0.7500
    Theoretical mean: 3.000000
    Theoretical variance: 48.000000
    Sample mean: 3.022224
    Sample var: 48.877018
--- theta = 0.9000
    Theoretical mean: 9.000000
    Theoretical variance: 900.000000
    Sample mean: 8.982110
    Sample var: 915.935810


## RGP(q = 2) distribution

In [12]:
print("RGP distribution")
sim_dist_thetas(log_a_rgp(2), log_phi_rgp(2), sup_rgp(2), [0.1, 0.2, 0.3, 0.4, 0.49], n, E_rgp, Var_rgp, q = 2)

RGP distribution
--- theta = 0.1000
    Theoretical mean: 0.125000
    Theoretical variance: 0.195312
    Sample mean: 0.125670
    Sample var: 0.195569
--- theta = 0.2000
    Theoretical mean: 0.333333
    Theoretical variance: 0.925926
    Sample mean: 0.333190
    Sample var: 0.920366
--- theta = 0.3000
    Theoretical mean: 0.750000
    Theoretical variance: 4.687500
    Sample mean: 0.755552
    Sample var: 4.811781
--- theta = 0.4000
    Theoretical mean: 2.000000
    Theoretical variance: 50.000000
    Sample mean: 2.009606
    Sample var: 50.397726
--- theta = 0.4900
    Theoretical mean: 24.500000
    Theoretical variance: 61250.000000
    Sample mean: 23.194992
    Sample var: 44644.666246


## RGP(q = -1/10) distribution

In [13]:
print("RGP distribution")
sim_dist_thetas(log_a_rgp(-1/10), log_phi_rgp(-1/10), sup_rgp(-1/10), [0.5, 1, 2, 5, 9], n, E_rgp, Var_rgp, q = -1/10)

RGP distribution
--- theta = 0.5000
    Theoretical mean: 0.476190
    Theoretical variance: 0.431919
    Sample mean: 0.475164
    Sample var: 0.430411
--- theta = 1.0000
    Theoretical mean: 0.909091
    Theoretical variance: 0.751315
    Sample mean: 0.910256
    Sample var: 0.752382
--- theta = 2.0000
    Theoretical mean: 1.666667
    Theoretical variance: 1.157407
    Sample mean: 1.665328
    Sample var: 1.157719
--- theta = 5.0000
    Theoretical mean: 3.333333
    Theoretical variance: 1.481481
    Sample mean: 3.337240
    Sample var: 1.481561
--- theta = 9.0000
    Theoretical mean: 4.736842
    Theoretical variance: 1.312145
    Sample mean: 4.737244
    Sample var: 1.315275


## Haight distribution

In [14]:
print("Haight distribution")
sim_dist_thetas(log_a_haight, log_phi_haight, sup_haight, [0.1, 0.2, 0.3, 0.4, 0.49], n, E_haight, Var_haight, q = None)

Haight distribution
--- theta = 0.1000
    Theoretical mean: 0.125000
    Theoretical variance: 0.175781
    Sample mean: 0.123364
    Sample var: 0.172969
--- theta = 0.2000
    Theoretical mean: 0.333333
    Theoretical variance: 0.740741
    Sample mean: 0.333424
    Sample var: 0.739948
--- theta = 0.3000
    Theoretical mean: 0.750000
    Theoretical variance: 3.281250
    Sample mean: 0.739742
    Sample var: 3.218868
--- theta = 0.4000
    Theoretical mean: 2.000000
    Theoretical variance: 30.000000
    Sample mean: 2.007000
    Sample var: 29.678999
--- theta = 0.4900
    Theoretical mean: 24.500000
    Theoretical variance: 31237.500000
    Sample mean: 24.485660
    Sample var: 30089.395458


## Geeta(q = 3) distribution

In [15]:
print("Geeta distribution (q = 3)")
sim_dist_thetas(log_a_geeta(3), log_phi_geeta(3), sup_geeta, [0.05, 0.1, 0.2, 0.3, 0.33], n, E_geeta, Var_geeta, q = 3)

Geeta distribution (q = 3)
--- theta = 0.0500
    Theoretical mean: 0.117647
    Theoretical variance: 0.154692
    Sample mean: 0.117814
    Sample var: 0.154878
--- theta = 0.1000
    Theoretical mean: 0.285714
    Theoretical variance: 0.524781
    Sample mean: 0.284858
    Sample var: 0.518946
--- theta = 0.2000
    Theoretical mean: 1.000000
    Theoretical variance: 5.000000
    Sample mean: 1.003358
    Sample var: 5.004619
--- theta = 0.3000
    Theoretical mean: 6.000000
    Theoretical variance: 420.000000
    Sample mean: 5.992862
    Sample var: 419.076271
--- theta = 0.3300
    Theoretical mean: 66.000000
    Theoretical variance: 442200.000000
    Sample mean: 51.928220
    Sample var: 144914.205368


In [16]:
# ---------------------------- Binomial Negativa (Mean-Variance parametrization) (MVNBcr) ----------------------------
# Versões sem o tensorflow
def log_a_mvnb(q):
    return lambda m : loggamma(1/q+m) - loggamma(1/q) - loggamma(m+1)
def log_phi_mvnb(q):
    return lambda theta : np.log(q*theta) - np.log(1+q*theta)
def C_mvnb(q):
    return lambda theta : (1 + q*theta)**(1/q)
def C_inv_mvnb(q):
    return lambda u : (u**q - 1)/q
# Versões para o tensorflow
def log_a_mvnb_tf(q):
    return lambda m : tf.math.lgamma(1/q+m) - tf.math.lgamma(1/q) - tf.math.lgamma(m+1)
def log_phi_mvnb_tf(q):
    return lambda theta : tf.math.log(q*theta) - tf.math.log(1+q*theta)
def C_mvnb_tf(q):
    return lambda theta : (1 + q*theta)**(1/q)
def C_inv_mvnb_tf(q):
    return lambda u : (u**q - 1) / q
sup_nb = np.arange(0, B, 1).astype(np.float64)

theta_min_nb = None
theta_max_nb = None
def E_mvnb(q, theta):
    return theta
# Overdispersion: in this case, variance is always greater than mean
def Var_mvnb(q, theta):
    return (1+q*theta)*theta

In [18]:
print("Mean-Variance Negative Binomial distribution (q = 2)")
sim_dist_thetas(log_a_mvnb(2), log_phi_mvnb(2), sup_nb, [0.22, 0.67, 2, 6, 18], n, E_mvnb, Var_mvnb, q = 2)

Mean-Variance Negative Binomial distribution (q = 2)
--- theta = 0.2200
    Theoretical mean: 0.220000
    Theoretical variance: 0.316800
    Sample mean: 0.218582
    Sample var: 0.314440
--- theta = 0.6700
    Theoretical mean: 0.670000
    Theoretical variance: 1.567800
    Sample mean: 0.665656
    Sample var: 1.548690
--- theta = 2.0000
    Theoretical mean: 2.000000
    Theoretical variance: 10.000000
    Sample mean: 2.009134
    Sample var: 10.081567
--- theta = 6.0000
    Theoretical mean: 6.000000
    Theoretical variance: 78.000000
    Sample mean: 6.025348
    Sample var: 78.356973
--- theta = 18.0000
    Theoretical mean: 18.000000
    Theoretical variance: 666.000000
    Sample mean: 17.979420
    Sample var: 664.204672
